# Seperating stock to forcast for certain dates depending on the product

In [ ]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn dart 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
import sys
sys.path.append('../../src')

In [ ]:
from data_config import ProductHandler as ph

In [ ]:
ph.custom_prod_set()

In [ ]:
# define the features and target variable from 'product_sales'
# CHANGE THE CUSTOMER CODE HERE >>>>>>>>>>
custom_code_df = pd.DataFrame(ph.get_custom_code_data('ALB'))

# customer code for later referencing
custom_ref = custom_code_df.loc[0, 'ProductNumber'][:3].lower().upper()
print(custom_ref)

custom_code_df.head()

In [ ]:
from app_utils import Transform as trans
from model_utils import *
from sklearn.pipeline import Pipeline

In [ ]:
# Compute OrderQuantity values with a zscore with threshold of 3 and remove them (Outlier removal)
custom_code_df = trans.compute_zscore(custom_code_df)

custom_code_df.head()

In [ ]:
X = custom_code_df.drop('OrderQuantity', axis=1)
y = custom_code_df.OrderQuantity

In [ ]:
# Trian and transforms data (CatBoost transforms natively so might not need to transform)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_preprocessed, X_val_preprocessed = trans.transform_data(X_train, X_val, X)

In [ ]:
nbeat_params = find_best_hyperparameters(NBEATSModel(), param_grids(NBEATSModel().__class__.__name__), X_train_preprocessed, y_train)

In [ ]:
nbeat_tuned = NBEATSModel(**nbeat_params)

In [ ]:
nbeat_tuned.fit(X_train, y_train)

print('----------- TRAINING METRICS -----------')
train_metrics = evaluate_model(nbeat_tuned, X_train, y_train)
print('\n\n')
print('----------- TESTING METRICS -----------')
test_metrics = evaluate_model(nbeat_tuned, X_val, y_val)